# Instalación e importación de librerías requeridas

In [ ]:
!pip install owslib # Manipulación de geoservicios
!pip install osmnx # Descarga de datos de OpenStreetMap

In [ ]:
import geopandas as gpd
from shapely import wkt
import pandas as pd
from owslib.wfs import WebFeatureService
import osmnx as ox

# Accediendo a geoservicios WFS

## Búsqueda de URL y petición GetCapabilities
Una fuente de direcciones URL a geoservicios es [Geoservicios IDERA.](https://www.idera.gob.ar/index.php?option=com_content&view=article&id=335:geoservicios&catid=33&Itemid=169) Una vez que accedemos y navegamos hacia la fuente de datos requeridas, copiamos el link de la columna WFS y la pegamos en la variable `wfs_url`.

In [ ]:
# URL del servicio WFS que deseas consultar
wfs_url = "https://wms.ign.gob.ar/geoserver/ows?service=wfs&version=1.1.0&request=GetCapabilities" # IGN

# Solicitud GetCapabilities al servicio WFS
wfs = WebFeatureService(wfs_url, version='1.0.0')

# Listado de capas publicadas
list(wfs.contents)

## Selección de capa y solicitud GetFeature
Seleccionamos de la lista un nombre y lo copiamos en la variable `capa`.

In [ ]:
# Obtener la capa disponibles en el servicio WFS
capa = ['ign:provincia'] # Es importante agregarla entre corchetes

# Realiza una solicitud GetFeature para obtener los datos del servicio WFS
response = wfs.getfeature(typename=capa)

# Utilizamos Geopandas para convertir los datos en un GeoDataFrame
gdf = gpd.read_file(response)

# Muestra la información del GeoDataFrame
gdf.head()

In [ ]:
# Ploteamos
gdf.plot()

In [ ]:
# Consultamos su CRS
print(gdf.crs)

In [ ]:
# Evitemos posibles problemas asignandole un CRS adecuado
gdf = gdf.set_crs('EPSG:????') # De acuerdo a las coordenadas de la columna "geometry" ¿En qué EPSG/CRS se encuentra?
gdf.crs

In [ ]:
# Ploteamos
gdf.plot()

## Exportamos nuestros datos
Guardamos los datos descargados en formato shapefile

In [ ]:
# Creamos una carpeta "data_export" si no existe
import os
if not os.path.exists('data_export'):
    os.makedirs('data_export')
    
# Guardamos nuestro GeoDataFrame como un shapefile
gdf.to_file('data_export/datos_wfs.shp', overwrite=True)

# Descarga de datos desde OpenStreetMap
La descarga de datos de OSM con la librería OSMNX requiere contemplar el tipo de información que se desea. En caso de requerir puntos de interés, edificios o equipamiento urbano, utilizamos la búsqueda por etiquetas y el método `features_from_place`. Este nos devuelve objetos GeoDataFrame.

Por otro lado, si deseamos obtener la red vial, utilizamos el método `graph_from_place`, que nos devolverá un objeto de tipo grafo.

## Descarga de equipamiento urbano, puntos de interés, etc

### Búsqueda de etiquetas
La descarga de datos desde OSM con el metodo `features_from_place()` se basa en la búsqueda por etiquetas. Los nombres de etiquetas permitidas pueden ser consultadas en el siguiente link:

[https://wiki.openstreetmap.org/wiki/ES:Objetos_del_mapa](https://wiki.openstreetmap.org/wiki/ES:Objetos_del_mapa)

Al buscar etiquetas, es útil tener en cuenta la estructura jerárquica de las mismas. Algunas etiquetas pueden ser más generales (por ejemplo, "amenity") y otras más específicas (por ejemplo, "bar").

Puedes usar etiquetas generales para obtener una amplia gama de elementos: `etiqueta = {'amenity': True}`, o ser más específico utilizando etiquetas más detalladas: `etiqueta = {'amenity': "bar"}`.

### Descarga de equipamiento por nombre de lugar


In [ ]:
# Definimos el lugar del que deseas obtener los datos (puede ser una ciudad, una dirección, etc.)
nombre_lugar = "Rosario, Santa Fe, Argentina"

# Define el diccionario de etiquetas para los elementos que deseas obtener.
etiqueta = {
    'amenity': 'bar',     # Equipamiento: Bar
}

# Obtiene los elementos disponibles en el lugar
datos_osm = ox.features_from_place(nombre_lugar, tags=etiqueta)

# Mostramos la tabla
datos_osm.head()

In [ ]:
# Muestra el listado de elementos disponibles
datos_osm.plot()

In [ ]:
# Controlamos su CRS
datos_osm.crs

### Exportamos nuestros datos
El código a continuación transforma los datos y los exporta a geopackage. Veamoslo con atención:

La primera línea del código utiliza el método `apply()` de Pandas para aplicar una función a cada columna del GeoDataFrame. En esta función, se realiza un ajuste de los tipos de datos de las columnas para asegurarse de que sean compatibles con el formato Geopackage y evitar posibles errores de exportación.

La función `lambda` en `lambda c: c.astype(str) if c.name != "geometry" else c` se aplica a cada columna `c` del GeoDataFrame. Si el nombre de la columna es diferente de "geometry" (es decir, si no es la columna que contiene las geometrías), entonces se convierten todos los elementos de esa columna en texto utilizando el método `astype(str)`. Esto es útil para garantizar que todos los datos en esas columnas sean del tipo de dato `str` (cadena), ya que algunos datos descargados de OpenStreetMap pueden ser de diferentes tipos.

In [ ]:
# Preparación de datos
datos_osm = datos_osm.apply(lambda c: c.astype(str) if c.name != "geometry" else c, axis=0)

# Creamos una carpeta "data_export" si no existe
import os
if not os.path.exists('data_export'):
    os.makedirs('data_export')
    
# Guardamos nuestro GeoDataFrame como un shapefile
datos_osm.to_file('data_export/datos_puntos_osmnx.gpkg')

## Descarga de calles desde OSM

In [ ]:
# Definimos el lugar del que deseas obtener los datos (puede ser una ciudad, una dirección, etc.)
nombre_lugar = "Rosario, Santa Fe, Argentina"

# Definimos el tipo de red a descargar. Puede ser: "all_private" , "all" , "bike" , "drive" , "drive_service" , "walk"
tipo_red = "drive"
red = ox.graph_from_place(nombre_lugar, network_type=tipo_red)

El objeto `red` es del tipo `MultiDiGraph` de la librería `NetworkX`. Esto permite realizar análisis espaciales sobre la red, como cálculos de caminos mas cortos, isocronas, etc.
Visualizamos los datos de manera rápida y luego los exportamos a Geopackage.

In [ ]:
# plot the network, but do not show it or close it yet
fig, ax = ox.plot_graph(
    red,
    bgcolor="w",
    edge_color="#333333",
    edge_linewidth=0.5,
    node_size=1,
    node_color="red"
)

### Exportamos nuestros datos
La librería provee un método para exportar objetos `MultiDiGraph` a un archivo Geopackage.

In [ ]:
# Creamos una carpeta "data_export" si no existe
import os
if not os.path.exists('data_export'):
    os.makedirs('data_export')

#Exportamos en geopackage
ox.save_graph_geopackage(red, filepath="data_export/red_osmnx.gpkg")

# Cargando datos de Poblaciones.org

In [ ]:
# Carga de datos como CSV
df = pd.read_csv('data/Indicadores de hogares. Provincias, 2010.csv')
df.head()

Necesitamos transformar nuestra columna "Geometría en WKT" en una Geoseries y luego en un GeoDataFrame.
Para eso utilizamos la siguiente sentencia:

In [ ]:
# Convierte la columna que contiene la geometría WKT
df['geometry'] = df['Geometría en WKT'].apply(wkt.loads)

# Convertir el DataFrame en un GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')

In [ ]:
gdf.plot()